# Atlantic
The Content Testing System in SMS

In [17]:
from gsheets import Sheets
import pandas as pd
import numpy as np
from tabulate import tabulate
import datetime as dt
from datetime import date
from datetime import timedelta
import warnings
import smartsheet
import infrastructure
import pygsheets
from scipy.stats import zscore
import filepaths
from datetime import datetime, timedelta
import random
import os
import filepath
import infrastructure

bold = "\033[1m"

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

### Read in Files

<strong>Requirements</strong>
- Lexi
- EIMT
- Offer_WK
- Cobra (mamba)


In [18]:
######################### - Getting Lexi - #########################
def reading_lexi():
    global lexi
    print("Reading in SMS Lexi...")
    sms_lexi = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
    lexi = infrastructure.transform_sms_df(sms_lexi)
    lexi['Hitpath Offer ID'] = lexi['Hitpath Offer ID'].astype(str).str.replace('.0', '')
    df = lexi
    return print("SMS Lexi is loaded.")

######################### - Getting Smartsheet - #########################
def reading_smartsheet():
    print("Gathering EIMT and Offer Workbook")
    global EMIT
    global offer_wk
    # use smartsheet api to get offer workbook and EIMT
    EMIT = infrastructure.get_publisher()
    offer_wk = infrastructure.get_smartsheet('offers_sms')
    offer_wk['Hitpath Offer ID'] = offer_wk['Hitpath Offer ID'].astype(str).str.replace('.0', '')
    return print("Smartsheets are loaded.")
def reading_cobra():
    global cobra
    cobra = infrastructure.get_mamba_full_slot()
    cobra['Hitpath Offer ID'] = cobra['Hitpath Offer ID'].astype(str).str.replace('.0', '')
    return print("Mamba are loaded.")


In [19]:
reading_lexi()
reading_smartsheet()
reading_cobra()
lexi = lexi.merge(offer_wk[['Hitpath Offer ID','Vertical']], left_on = 'Hitpath Offer ID',right_on = 'Hitpath Offer ID', how = 'left')

Reading in SMS Lexi...
SMS Lexi is loaded.
Gathering EIMT and Offer Workbook
Smartsheets are loaded.
Mamba are loaded.


In [20]:
mamba_dic_df = infrastructure.get_mamba_directory()
mamba_dic_df['Text ID'] = mamba_dic_df['Account'].str.split('_', expand=True)[2]
current_active_segment = mamba_dic_df['Segment'].str.split('\n').tolist()
current_active_segment = [item for sublist in current_active_segment for item in sublist if item]
current_active_sc_sv = mamba_dic_df['Account'].str[:-7].unique().tolist()
#EMIT["Dataset"] = EMIT["Text ID"].str.cat(EMIT["Text DP"], sep = "_")
#DPPub_to_Dataset = dict(zip(EMIT['shortcode_DP.SV'], EMIT['Dataset']))
#Dataset_to_DPPub = dict(zip(EMIT['Dataset'], EMIT['shortcode_DP.SV']))
print(current_active_sc_sv)

['MBC_NPD.RTO', 'FLC_NPD.RTO', 'MBC_I.RC', 'FLC_I.RC', 'HZB_CM.OSR', 'MBC_CM.OSR', 'DSS_DMS.SJ', 'HZB_DMS.SJ', 'SVT_AMD.PL', 'HZB_AMD.PL', 'SVT_AL.PL.3', 'HZB_AL.PL.3', 'A4F_I.A4F', 'DSS_TLG.PL', 'HZB_TLG.PL', 'HZB_PN.SWP', 'MBC_PN.SWP', 'FLC_PN.SWP', 'UAA_SPK.SWP2', 'UAATF_SPK.SWP2', 'MBC_PA.PS', 'HZB_PA.PS', 'UAA_FSM.YS', 'HZB_FSM.YS', 'FLC_EDM.247L', 'MBC_EDM.247L', 'DSS_ZM.PL.2', 'HZB_ZM.PL.2', 'MFA_I.MFA', 'SVT_B2.F', 'HZB_B2.F', 'MBC_WWM.YFA.2', 'FLC_WWM.YFA.2', 'FLC_I.CC', 'MBC_I.CC', 'HZB_RHD.CC', 'DSS_RHD.CC', 'UAA_SPK.CR2', 'FLC_SPK.CR2', 'MBC_PN.FC', 'FLC_PN.FC', 'SVT_AL.PL', 'HZB_AL.PL', 'SVT_AL.PL.2', 'HZB_AL.PL.2', 'DSS_JET.ZTA', 'FLC_JET.ZTA']


# Functions

In [21]:
def check_CT_eligiblity(default_max_drops, offer, lexi, cobra):
    """ 
    #sheets = Sheets.from_files(filepaths.gsheets)
    #viper_settings = sheets.get(CT_settings_url)

    #CT_settings = viper_settings.find('Testing Settings').to_frame()
    #CT_settings = CT_settings.fillna(default_max_drops) # MAX NUMBER OF P DROPS IS DEFAULTED TO 2
    #CT_settings_dict = dict(zip(CT_settings['shortcode_DP.SV'],CT_settings['Max Number of CT']))
    """
    CT_settings_dict = {}
    for i in current_active_sc_sv: 
        CT_settings_dict[i] = default_max_drops
    # Dates and Parameter Stuff
    ## Calculate the starting date of the current week
    current_date = datetime.today()
    start_of_week = datetime.today() - timedelta(days=current_date.weekday() + 1)
    start_date = datetime.today()
    end_date = start_date + timedelta(14)
    
    # Manipulating Cobra
    #cobra['shortcode_DP.SV'] = cobra['Dataset'].map(dataset_convertor)
    cobra_temp = cobra.groupby([pd.Grouper(key='Date', freq='W'), 'shortcode_DP.SV'])['Send Strategy'].count().reset_index()
    cobra_temp = cobra_temp[['shortcode_DP.SV', 'Date']]
    Account_CT_Checklist = cobra_temp[cobra_temp['Date'] >= (start_of_week - timedelta(1))].sort_values(by = ['shortcode_DP.SV', 'Date'], ascending = True)
    #Account_CT_Checklist = pd.DataFrame({'shortcode_DP.SV': cobra_temp[cobra_temp['Date'] >  dt.date.today()]['shortcode_DP.SV'].unique()})
    cobra_CT = cobra[cobra['Send Strategy'] == 'CT']

         # use pd.Grouper to get current and future week
    cobra_weekly_CT = cobra_CT.groupby([pd.Grouper(key='Date', freq='W'), 'shortcode_DP.SV'])['Send Strategy'].count().reset_index()
    cobra_weekly_CT.rename(columns={'Send Strategy': 'CT Count'}, inplace=True)
    cobra_weekly_CT = cobra_weekly_CT[cobra_weekly_CT['Date'] > (start_of_week - timedelta(1))]
    cobra_weekly_CT['Max CT'] = cobra_weekly_CT['shortcode_DP.SV'].map(CT_settings_dict)
    cobra_weekly_CT['CT available'] = cobra_weekly_CT['Max CT'] - cobra_weekly_CT['CT Count']
    Account_CT_Checklist = pd.merge(Account_CT_Checklist, cobra_weekly_CT, on=['shortcode_DP.SV','Date'], how='left')
    # earliest possible date
    default_earliest_date = dt.date.today() + timedelta(2)
    
# See if account has a P drop to swap and is under its Max CT, if so, swap the P drop to a CT
    cobra_P = cobra[cobra['Send Strategy'] == 'P']
    cobra_P = cobra_P[cobra_P['Hitpath Offer ID'] == offer]
    #cobra_P = cobra_P[cobra_P['Date'].dt.date >= default_earliest_date]
    cobra_weekly_P = cobra_P.groupby([pd.Grouper(key='Date', freq='W'), 'shortcode_DP.SV'])['Send Strategy'].count().reset_index()
    cobra_weekly_P.rename(columns={'Send Strategy': 'P Available to Swap to CT'}, inplace=True)
    cobra_weekly_P = cobra_weekly_P[cobra_weekly_P['Date'] > (start_of_week - timedelta(1))]
    Account_CT_Checklist = pd.merge(Account_CT_Checklist, cobra_weekly_P, on=['shortcode_DP.SV','Date'], how='left')

# If not, but is under Max CT and has sent the offer more than 1 P drop in last 30 days, send in P slot
## Can send in normal P slot?
    date_threshhold = max(lexi['Date']) - timedelta(30)
    lexi_30 = lexi[lexi['Date'] > date_threshhold]
    lexi_30 = lexi_30[lexi_30['Send Strategy'] == 'P']
    lexi_30 = lexi_30[lexi_30['Hitpath Offer ID'] == offer]
    lexi_30 = lexi_30.groupby(['shortcode_DP.SV'])['Send Strategy'].count().reset_index()
    lexi_30.rename(columns={'Send Strategy': 'Sent in Last 30 Days'}, inplace=True)
    lexi_30['Can it Send in P Slot?'] = lexi_30['Sent in Last 30 Days'].apply(lambda x: 1 if x > 0 else 0)
    Account_CT_Checklist = pd.merge(Account_CT_Checklist, lexi_30[['shortcode_DP.SV', 'Can it Send in P Slot?']], on=['shortcode_DP.SV'], how='left')

# If not, but is under max CT and has not send the offer more than once in last 30 days, send outside P slot
## Needs to send outside normal P slots?

    
# If account has not sent the offer at all


# Cleaning up NaNs
    Account_CT_Checklist['CT Count'].fillna(0, inplace=True)
    Account_CT_Checklist['P Available to Swap to CT'].fillna(0, inplace=True)
    Account_CT_Checklist['Can it Send in P Slot?'].fillna(0, inplace=True)
    Account_CT_Checklist['Max CT'] = Account_CT_Checklist['shortcode_DP.SV'].map(CT_settings_dict)
    Account_CT_Checklist['CT available'] = Account_CT_Checklist['Max CT'] - Account_CT_Checklist['CT Count']
     
    '''
    print("The start date:", start_date.strftime("%Y-%m-%d"))
    print("The end date:", end_date.strftime("%Y-%m-%d"))
    print("The start date of the current week is:", start_of_week.strftime("%Y-%m-%d"))
    '''
    
    return Account_CT_Checklist


In [22]:
def get_offer_vertical_gaps(df):
    df = lexi.copy()
    df.sort_values(by = ['shortcode_DP.SV','Hitpath Offer ID','Send Strategy','Date'], ascending = True, inplace = True)
    df['Offer Gap'] = df.groupby(['shortcode_DP.SV','Send Strategy','Hitpath Offer ID'])['Date'].diff()
    df['Offer Gap'] = df['Offer Gap'].dt.days
    df.loc[df['Send Strategy'] != 'P', 'Offer Gap'] = np.nan
    # filter for last 60 days
    date_threshhold = max(df['Date']) - timedelta(60)
    df = df[df['Date'] > date_threshhold]
    
    #df = df.rename(columns={"Vertical": "Offer Vertical"})
    df_gap = df[(df['Offer Gap'] > 0) & (df['Offer Gap'] < 30)]
    df_gap = df_gap.groupby(['Data Vertical','Offer Vertical'])['Offer Gap'].mean().reset_index()
    return df_gap
    
    

In [23]:
def get_account_rankings(df, offer,cobra):
    
        # offer info
    
    #offer_vertical = offer_wk[offer_wk['Hitpath Offer ID'] == offer]['Vertical'].iloc[0]
    
    filtered_df = offer_wk[offer_wk['Hitpath Offer ID'] == offer]

    # check if the filtered DataFrame is not empty
    if not filtered_df.empty:
        offer_vertical = filtered_df['Vertical'].iloc[0]
    else:
        offer_vertical = None
    
    print(offer, offer_vertical)
        
        # filter for last 60 days
    date_threshhold = max(df['Date']) - timedelta(60)
    df = df[df['Date'] > date_threshhold]
    
        # get account ranking for offer
    df_offer = df[df['Hitpath Offer ID'] == offer]
    df_offer = df_offer.groupby('SC_DP&Pub')[['Revenue', 'Delivered', 'Clicks', 'Jump Page Clicks', 'Opportunity Cost']].sum().reset_index()
    df_offer['eCPM'] = 1000 * (df_offer['Revenue']/df_offer['Delivered'])
    df_offer['OCRR'] = (df_offer['Opportunity Cost']/df_offer['Revenue'])
        # Create rank columns for each column
    rank_columns = df_offer[['Revenue', 'Delivered','eCPM', 'Opportunity Cost','OCRR']].apply(lambda x: x.rank(ascending=False))
        # Add rank columns to the original DataFrame
    df_offer = pd.concat([df_offer, rank_columns.add_suffix('_rank')], axis=1)
    df_offer['Average Rank'] = df_offer.iloc[:,8:].mean(axis=1)
    df_offer = df_offer.sort_values(by = 'Average Rank', ascending = True)
    df_offer['Overall Rank'] = df_offer['Average Rank'].rank()
    
        # offer vertical ranking
    df_vertical = df[df['Vertical'] == offer_vertical]
    df_vertical = df_vertical.groupby('SC_DP&Pub')[['Revenue', 'Delivered', 'Clicks','Jump Page Clicks', 'Opportunity Cost']].sum().reset_index()
    df_vertical['eCPM (OV)'] = 1000 * (df_vertical['Revenue']/df_vertical['Delivered'])
    df_vertical['OCRR (OV)'] = (df_vertical['Opportunity Cost']/df_vertical['Revenue'])
        # Create rank columns for each column
    rank_columns = df_vertical[['Revenue', 'Delivered','eCPM (OV)', 'Opportunity Cost','OCRR (OV)']].apply(lambda x: x.rank(ascending=False))
        # Add rank columns to the original DataFrame
    df_vertical = pd.concat([df_vertical, rank_columns.add_suffix('_rank (OV)')], axis=1)
    df_vertical['Average Rank (OV)'] = df_offer.iloc[:,8:].mean(axis=1)
    df_vertical = df_vertical.sort_values(by = 'Average Rank (OV)', ascending = True)
    df_vertical['Overall Rank'] = df_vertical['Average Rank (OV)'].rank()
    df_vertical['Overall Rank'] = df_vertical['Overall Rank'].apply(lambda x: x + 5 if not pd.isnull(x) else x)
    df_vertical = df_vertical[['SC_DP&Pub','Overall Rank']]
    df_vertical['Rank Origin'] = 'Vertical'
    
        # Create column that checks eligibility
    #static_eligbility = check_ESP_eligibility(df=EMIT)
    #df_offer['CT Eligibility'] = df_offer['shortcode_DP.SV'].isin(static_eligbility).astype(int)
    #df_offer_eligble = df_offer[df_offer['CT Eligibility'] == 1]
    df_offer_eligble = df_offer.copy()
    
        # calculate Tiers
        # Replace -inf with NaN
    df_offer_eligble['OCRR'] = df_offer_eligble['OCRR'].replace(float('-inf'), np.nan)
        # Calculate the z-scores
    df_offer_eligble['OCRR Z-Scores'] = zscore(df_offer_eligble['OCRR'], nan_policy='omit')
    
    Tier_1 = .5
    Tier_2 = .25
    Tier_3 = 0
    Tier_4 = -.25
    
    conditions = [
        (df_offer_eligble['OCRR Z-Scores'] >= Tier_1),
        (df_offer_eligble['OCRR Z-Scores'] < Tier_1) &  (df_offer_eligble['OCRR Z-Scores'] >= Tier_2),
        (df_offer_eligble['OCRR Z-Scores'] < Tier_2) &  (df_offer_eligble['OCRR Z-Scores'] >= Tier_3),
        (df_offer_eligble['OCRR Z-Scores'] < Tier_3) &  (df_offer_eligble['OCRR Z-Scores'] >= Tier_4),
        (df_offer_eligble['OCRR Z-Scores'] < Tier_4)
    ]
    
    message = ['Tier 1', 'Tier 2', 'Tier 3',
               'Tier 4','Tier 5']
    
    df_offer_eligble['Account Tier'] = np.select(conditions, message)
    df_tier_summary = pd.DataFrame(df_offer_eligble['Account Tier'].value_counts())
    df_tier_summary['Tier'] = df_tier_summary.index
    df_tier_summary.rename(columns={'Account Tier': 'Accounts per Tier'}, inplace=True)
    df_tier_summary = df_tier_summary[['Tier', 'Accounts per Tier']].reset_index(drop = True)
    df_tier_summary = df_tier_summary.sort_values(by = 'Tier', ascending = True)
    df_tier_summary = tabulate(df_tier_summary, headers='keys', tablefmt='grid')
    
    #print("For Hitpath Offer ID", offer, "the account Tier distribution is: \n", df_tier_summary )
    
    accounts_ranked = df_offer_eligble[['SC_DP&Pub','Overall Rank']].sort_values(by = ['Overall Rank'], ascending = True)
    accounts_ranked['Rank Origin'] = 'Account'
    
    accounts_ranked = pd.concat([accounts_ranked, df_vertical], ignore_index=True)
    accounts_ranked = accounts_ranked[~((accounts_ranked['Rank Origin'] == 'Vertical') & accounts_ranked.duplicated('SC_DP&Pub'))]
    accounts_ranked['PubID'] = accounts_ranked['SC_DP&Pub'].str[-6:]
    accounts_ranked['shortcode_DP.SV'] = accounts_ranked['SC_DP&Pub'].str[:-7]
    
    # filter out accounts that are paused to the offer
    paused_pubs = offer_wk[offer_wk['Hitpath Offer ID'] == offer]['Paused Pubs'].str.split('\n|\t|,|"| ')
    accounts_ranked = accounts_ranked[~accounts_ranked['PubID'].isin(paused_pubs)]

    # give account has a CT slot available some extra credit 
    # earliest possible date
    default_earliest_date = dt.date.today() + timedelta(2)
    cobra_P = cobra[cobra['Send Strategy'] == 'P']
    cobra_P = cobra_P[cobra_P['Hitpath Offer ID'] == offer]
    cobra_weekly_P_pubid = cobra_P[cobra_P['Date'].dt.date >= default_earliest_date]['shortcode_DP.SV'].unique().tolist()
    accounts_ranked['CT Slot Available'] = accounts_ranked['shortcode_DP.SV'].isin(cobra_weekly_P_pubid).astype(int)
    accounts_ranked['CT Slot Rank'] = 1- accounts_ranked['CT Slot Available']
    accounts_ranked['Overall Rank'] = accounts_ranked['Overall Rank'] + accounts_ranked['CT Slot Rank']
    accounts_ranked = accounts_ranked.sort_values(by = ['Overall Rank','CT Slot Rank'], ascending = True)
    return accounts_ranked



In [24]:
def adjust_date(date, offer):
    
    days_restrictions = offer_wk[offer_wk['Hitpath Offer ID'] == offer]['Day Restrictions'].iloc[0]
    days_allowed = str(days_restrictions)
    
    # Parse the input date
    #input_date = pd.Timestamp(date)
    input_date = date
    
    # If days_allowed is None or 'None', return the input date as it is
    if days_allowed is None or days_allowed.lower() == 'none' or days_allowed.lower() == 'nan':
        return pd.Timestamp(input_date)

    # Create a dictionary to map weekdays to numbers (Monday = 0, Tuesday = 1, etc.)
    weekdays = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

    # Check if the days_allowed contains a single value or a range
    if ' - ' in days_allowed:
        lower_bound, upper_bound = days_allowed.split(' - ')
    else:
        lower_bound = upper_bound = days_allowed

    # Calculate the target weekday numbers for the lower and upper bounds
    target_lower = weekdays[lower_bound]
    target_upper = weekdays[upper_bound]

    # Calculate the current weekday number of the input date
    current_weekday = input_date.weekday()

    # Calculate the difference between the current weekday and the target lower bound
    weekday_difference = (target_lower - current_weekday) % 7

    # Adjust the input date by adding the weekday difference
    adjusted_date = input_date + timedelta(days=weekday_difference)

    # If the adjusted date is still outside the days allowed, add 1 day until it falls within the allowed range
    while adjusted_date.weekday() < target_lower or adjusted_date.weekday() > target_upper:
        adjusted_date += timedelta(days=1)

    # Return the adjusted date as a string in the format 'MM/DD/YYYY'
    return pd.Timestamp(adjusted_date)

In [25]:
def get_drop_date(offer_gaps_df,CT_eligibility_df,account, offer, cobra, end_date):
    print("#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#")
    print("Searching for available date for", account)
    
    cobra = cobra[cobra['Send Strategy'] == 'P']
    
    all_drop_dates = cobra[['shortcode_DP.SV','Hitpath Offer ID', 'Date']]
    all_drop_dates = all_drop_dates[all_drop_dates['Hitpath Offer ID'] == offer]
    all_drop_dates = all_drop_dates[all_drop_dates['shortcode_DP.SV'] == account]
    all_drop_dates['Date'] = pd.to_datetime(all_drop_dates['Date'])
    
    last_drop_dates = cobra.groupby(['shortcode_DP.SV','Hitpath Offer ID'])['Date'].max().reset_index()
    last_drop_dates = last_drop_dates[last_drop_dates['Hitpath Offer ID'] == offer]
    last_drop_dates = last_drop_dates[last_drop_dates['shortcode_DP.SV'] == account]
    
    account_dv = lexi[lexi['shortcode_DP.SV'] == account]['Data Vertical'].iloc[0]
    offer_vertical = offer_wk[offer_wk['Hitpath Offer ID'] == offer]['Vertical'].iloc[0]
    offer_gap = offer_gaps_df[offer_gaps_df['Data Vertical'] == account_dv]
    offer_gap_df = offer_gap[offer_gap['Offer Vertical'] == offer_vertical]
    
    if len(offer_gap_df) == 0:
        offer_gap = 10
    elif len(offer_gap_df) != 0:
        offer_gap = offer_gap_df['Offer Gap'].iloc[0].round(0)
    else:
        offer_gap = 10

    # earliest possible date
    default_earliest_date = dt.date.today() + timedelta(3)
    
    # next upcoming drop based on offer gap
    if len(last_drop_dates) == 0:
        next_drop_date = default_earliest_date
    if len(last_drop_dates) != 0:
        next_drop_date = last_drop_dates['Date'].iloc[0] +  timedelta(offer_gap)
    
    # first week it's avaible to drop
    # check the eligibility of accounts
    CT_eligibility_Pub = CT_eligibility_df[CT_eligibility_df['shortcode_DP.SV'] == account]
    CT_eligibility_Pub = CT_eligibility_Pub[CT_eligibility_Pub['CT available'] > 0 ]
    
    if CT_eligibility_Pub['P Available to Swap to CT'].iloc[0] == 1:
        
        earliest_week = CT_eligibility_Pub['Date'].iloc[0]
        earliest_week = max(pd.to_datetime(default_earliest_date), 
                   pd.to_datetime(earliest_week)
                  )
        input_date = pd.to_datetime(earliest_week)
        if len(all_drop_dates[all_drop_dates['Date'] > input_date]['Date']) > 0:
            print("     There is an existing P drop, will swap.")
            next_drop_date = pd.to_datetime(all_drop_dates[all_drop_dates['Date'] > input_date]['Date'].min())
            print("     Swapping drop on", next_drop_date)
        else: 
            print("     No existing P drop, we will add in a drop.")
            print("     The earliest available week to test with proper gapping is", next_drop_date)

    
    if (CT_eligibility_Pub['P Available to Swap to CT'].iloc[0] != 1):
        print("     No existing P drop, we will add in a drop.")
        earliest_week = CT_eligibility_Pub['Date'].iloc[0]
        print("     The earliest available week to test with proper gapping is", next_drop_date)

    the_date = max(pd.to_datetime(default_earliest_date), 
                   pd.to_datetime(next_drop_date), 
                   pd.to_datetime(earliest_week)
                  )
    
    # check to make sure it's on a weekday and not on a restricted day
    the_date = adjust_date(the_date, offer)

    print("     Earliest available drop is", the_date)
    
    if the_date.date() >= dt.date.today()+ timedelta(end_date):
        print("     This account is NOT eligible to CT this offer.")
        print(" ")
        result = 0
    elif the_date.date() < dt.date.today() + timedelta(end_date):
        print("     This account is eligible to CT this offer.")
        print(" ")
        result = 1       
    return the_date, result
    

In [26]:
# at the moment, we don't have content feedback report, I need to import Lanina and find out the content that is waiting for test. 
def read_content_feedback_report(directory_path):

    # List all files and directories in the specified directory
    contents = os.listdir(directory_path)
    
    today = date.today().strftime("%m_%d_%Y")
    filename_date = f'Content_Feedback_Loop_Report_{today}.xlsx' 

    df = pd.DataFrame(contents, columns=['File Name'])
    filtered_df = df[df['File Name'].str.contains(filename_date)]
    filtered_df['Date'] = filtered_df['File Name'].str.extract(r'(\d{2}_\d{2}_\d{4})')
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'], format='%m_%d_%Y')
    filtered_df = filtered_df.sort_values(by = 'Date', ascending = False)
    file_name = str(filtered_df['File Name'].iloc[0])
    
    content_feedback_report = pd.read_excel(directory_path+file_name, sheet_name = 'Awaiting Testing -  Live')

    return content_feedback_report

In [27]:
def get_max_P_drops(df):

    # Get the current date
    current_date = datetime.today()

    # Calculate the date 1 week prior
    one_week_prior = current_date - timedelta(days=7)

    # Filter the dataframe to include only rows within the date range and where the 'Send Strategy' column contains 'P'
    filtered_df = df[(df['Date'] >= one_week_prior) & (df['Date'] <= current_date) & (df['Send Strategy'] == 'P')]

    # Group the filtered dataframe by 'Date' and count the occurrences of 'P' per day
    p_counts = filtered_df.groupby(['shortcode_DP.SV','Date'])['Send Strategy'].count().reset_index()
    average_frequency = p_counts.groupby(['shortcode_DP.SV'])['Send Strategy'].mean().reset_index()

    # Calculate the average frequency of 'P' per day
    average_frequency['Send Strategy'] = np.floor(average_frequency['Send Strategy'])
    
    Account_P_Max = dict(zip(average_frequency['shortcode_DP.SV'], average_frequency['Send Strategy']))

    return Account_P_Max
def choose_drop_slot(cobra, account, date, offer, P_drop_dict, eligibility_df):

    # get schedule that day in that account
    cobra_pub = cobra[cobra['shortcode_DP.SV'] == account]
    cobra_pub = cobra_pub[cobra_pub['Date'] == date]
    
    # get max P drop for the account
    Min_P = 1
    Max_P = int(P_drop_dict.get(account, 3))
    all_P_slots = list(range(2, 10+1))
    restricted_p_slots = list(range(2, Max_P+2))

    # if drop exists on that day, overwrite
    cobra_pub_offer = cobra_pub[cobra_pub['Hitpath Offer ID'] == offer]
    if len(cobra_pub_offer) > 0:
        drop_slot = cobra_pub_offer['Drop']

    elif len(cobra_pub_offer) < 1:

        """ 
        # if P eligbility is 1, then it can test within normal P slots
        # In email: if the offer we sent in the last 30 days, we think the CT drops can directly be sent in the P drops, otherwise, we will send it extra drop outside the maximum P drops
        # In sms: If the offer we didn't schedule but need CT, we consider everything as normal P drops (Offset and limit a test drop in drop 2)
        if eligibility_df['Can it Send in P Slot?'].iloc[0] == 1:
            cobra_pub_2 = cobra_pub[cobra_pub['Hitpath Offer ID']=='nan']
            cobra_pub_2 = cobra_pub_2.sort_values(by = 'Drop', ascending = True)
            open_drops = list(cobra_pub_2['Drop'])
            open_drops = list(map(int, open_drops)) 
            drop_slot = min([x for x in open_drops if x >= Min_P])
        
        # if P eligibility is 0, then it will test outside normal P slots
        if eligibility_df['Can it Send in P Slot?'].iloc[0] == 0:
            cobra_pub_2 = cobra_pub[cobra_pub['Hitpath Offer ID']=='nan']
            cobra_pub_2 = cobra_pub_2.sort_values(by = 'Drop', ascending = True)
            open_drops = list(cobra_pub_2['Drop'])
            open_drops = list(map(int, open_drops))
            drop_slot = min([x for x in open_drops if x > Max_P])
        """
        cobra_pub_2 = cobra_pub[((cobra_pub['Hitpath Offer ID']=='nan')| (cobra_pub['Hitpath Offer ID'] == '')) & (cobra_pub['Drop'] == 2)]
        if len(cobra_pub_2) > 0:
            drop_slot = 2
        else: 
            print("The day is full with Test drop")

        # if neither 1 or 0 somehow, then test outside normal P slots

    else:
        print("help me pls")


    return int(drop_slot)


    

In [28]:
def get_cobra_slot(cobra, account, date, offer, Drop_Number, CCIDs):
    global limit_size
    result = 1
    cobra_slot = cobra[(cobra['shortcode_DP.SV'] == account) & (cobra['Date'] == date) & (cobra['Drop'] == 1)]
    sc = account.split('_')[0]
    # Time - check orginal, if not, then use default
    time = cobra_slot['Time'].values[0]
    if time == '':
        time = cobra[(cobra['shortcode_DP.SV'] == account) & (cobra['Time']!='') & (cobra['Drop'] == 1)].sort_values(by = 'Date', ascending = False)['Time'].values[0]
    # Segment - unified 
    segment = cobra_slot['Segment '].values[0]
    if segment == '':
        segment = cobra[(cobra['shortcode_DP.SV'] == account) & (cobra['Segment ']!='') & (cobra['Drop'] == 1)].sort_values(by = 'Date', ascending = False)['Segment '].values[0]
    # Offer - unified
    ss_offer = infrastructure.get_ss_offer()
    try: 
        campaign = ss_offer.loc[(ss_offer['SS Offers (updated)'].str.contains(offer, na = False)) & (ss_offer['SS Offers (updated)'].str[-4:].str.contains(sc, na = False)), 'SS Offers (updated)' ].values[0]
    except: 
        print("The offer is not added in the SS.")
    # Job Name - unified 
    date1 = date.strftime("%d%b%y")  
    job_name = "SS_"+segment[:3] + "_"+segment[4:].replace(".",'-').replace("_",'-')+"_"+ offer +"_"+ "CT" + "_" +   date1
    # Offset - seperate, keep first one from original 
    # Creative - seperate, keep first one from original
    if Drop_Number == 1:
        limit = cobra_slot['Limit'].values[0]
        offset  =  cobra_slot['Offset'].values[0]
    else: 
        get_limit_size = limit_size(offer_id_test = offer ,df = lexi,account = account)
        result = get_limit_size[1] 
        limit = get_limit_size[0]
        offset = 0 
    # Creative 
    creative = cobra_slot['Creative'].values[0]
    if (creative !='') & (Drop_Number == 1):
        creative = creative.split('\n')[0] + "\n" + CCIDs
    else: 
        creative = CCIDs
    
    return time, segment, campaign, limit,offset, creative,job_name,result 
        


 

In [29]:
# Define limit and offset size if we want to add a new CT drop 
def limit_size(offer_id_test,df,account):
    #input
    test_offer_info = offer_wk.loc[offer_wk['Hitpath Offer ID'] == offer_id_test]
    payout = float(test_offer_info['$ Payout'].replace(r'[^0-9\.]','',regex=True))
    vertical = offer_wk.loc[offer_wk['Hitpath Offer ID'] == offer_id_test, 'Vertical'].iloc[0]
    if(payout  > 50.00):
        payout = payout/2
    # payout = 8.00
    
    expected_revenue = payout*1.6+20
    #last 60 days of data
    last60days= pd.Timestamp(dt.date.today()-dt.timedelta(days=60)).strftime('%Y-%m-%d')
    df = df[df['Date']>=last60days]
    df['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype(str)
    df['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)
    df = df[df['Send Strategy'] != 'AR']
    #median delivered volume by shortcode_DP.SV
    median_delivered = df.groupby(['Date','shortcode_DP.SV'])['Delivered'].sum().reset_index()
    median_delivered = median_delivered.groupby('shortcode_DP.SV')['Delivered'].median().reset_index()
    #eCPM by shortcode_DP.SV
    eCPM_df = df.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'})
    eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']
    #merge eCPM and median deliverd
    merged_segment_size = median_delivered.merge(eCPM_df,how = 'right', on = 'shortcode_DP.SV')
    merged_segment_size = merged_segment_size.rename(columns=
                                                    {"Delivered_x": "Median Delivered", 
                                                    "Delivered_y": "Total Delivered"})
    merged_segment_size['segment_size'] = expected_revenue * 1000 / merged_segment_size['eCPM']
    merged_segment_size['segment_size'] -= merged_segment_size['segment_size'] % -100
    merged_segment_size['% Testing Size'] = merged_segment_size['segment_size']/ merged_segment_size['Median Delivered']
    merged_segment_size['% Testing Size'] = merged_segment_size['% Testing Size'].fillna(0)
    proper_segment = merged_segment_size.loc[(merged_segment_size['% Testing Size'] < 0.5),]
    if len(proper_segment) <2:
        proper_segment['segment_size'] = 4000
        proper_segment['% Testing Size'] = proper_segment['segment_size']/ proper_segment['Median Delivered']
        proper_segment = proper_segment.loc[(proper_segment['% Testing Size'] < 0.5),]
    limit = proper_segment.loc[(proper_segment['shortcode_DP.SV'] == account) &(proper_segment['% Testing Size'] < 0.5), 'segment_size'].values[0] 
    if limit is None :
        print("     This account is NOT eligible to CT this offer because of segment size.")
        print(" ")
        result = 0
    else:
        result = 1       
    return limit, result 


## Driver Code

In [30]:
reading_cobra()
cobra['Drop'] = cobra['Drop'].str.replace('Drop ', '').astype(int)
Account_P_Max = get_max_P_drops(df = cobra)
content_feedback_report = read_content_feedback_report(
                    directory_path = '/Users/garrettchao/Desktop/contentfeedbackloop/Content Feedback Loop Reports/')

content_feedback_report = content_feedback_report.rename(columns={'HitPath ID': 'Hitpath Offer ID'})




#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#- Functions -#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

print("Starting process to create CCID testing schedule.")

# create empty df to store schedule

columns = [
    'Date',
    'Affiliate ID_DP.DS',
    'Drop Number',
    'Time',
    'Segment',
    'Send Strategy',
    'Offer',
    'Limit',
    'Offset',
    'Creative',
    'Job Name',
    'Can it Test?'
]

# Create an empty DataFrame
upcoming_CT_schedule = pd.DataFrame()
content_feedback_report['Hitpath Offer ID'] = content_feedback_report['Hitpath Offer ID'].astype('str').str.split('.',expand = True)[0]
content_feedback_report = content_feedback_report[content_feedback_report['Hitpath Offer ID']!='nan']
for index, row in content_feedback_report.iterrows():
#for hitpath_offer_ID in content_feedback_report['Hitpath Offer ID']:

    # drop, CCID, hitpath info
    hitpath_offer_ID = row['Hitpath Offer ID']
    CCIDs_to_Test = row["Content ID's"]
    sc = row['Shortcode']
    #days_restrictions = offer_wk[offer_wk['Hitpath Offer ID'] == hitpath_offer_ID]['Day Restrictions'].iloc[0]
    
    print("------------------------------------------------------------------------")
    print("Generating Schedule for CCIDs using Hitpath Offer ID:", hitpath_offer_ID)
    print("------------------------------------------------------------------------")
    
    #offer_vertical = offer_wk[offer_wk['Hitpath Offer ID'] == hitpath_offer_ID]['Vertical'].iloc[0]
    
    filtered_df = offer_wk[offer_wk['Hitpath Offer ID'] == hitpath_offer_ID]

    # check if the filtered DataFrame is not empty
    if not filtered_df.empty:
        offer_vertical = filtered_df['Vertical'].iloc[0]
    else:
        offer_vertical = None
    
    if (offer_vertical == 'Loan') & (sc == 'UAA'):
        print("We can't test loan offer in UAA")
    else: 
        offer_gaps_df = get_offer_vertical_gaps(df = lexi)
        lexi_sc = lexi[lexi['Shortcode Name'] == sc]
        account_rankings_df = get_account_rankings(df = lexi_sc, offer = hitpath_offer_ID,cobra = cobra)
        DPPub_test = list(account_rankings_df['shortcode_DP.SV'])
        
        
        success_counter = 0
        for DPPub in DPPub_test:

            try:
                CT_eligibility_df = check_CT_eligiblity(
                                                    default_max_drops = 2,
                                                    lexi = lexi,
                                                    cobra = cobra,
                                                    offer = hitpath_offer_ID)
                drop_date_info = get_drop_date(offer_gaps_df = offer_gaps_df,
                                            CT_eligibility_df = CT_eligibility_df,
                                                account = DPPub, 
                                            offer = hitpath_offer_ID, 
                                            cobra = cobra, 
                                            end_date = 14)
                drop_date = drop_date_info[0]
                
                # drop info to put into auto scheduler
                Date_output = drop_date
                DPPub_output = DPPub
                
                Drop_Number_output = choose_drop_slot(cobra = cobra,
                                                    account = DPPub,
                                                    date = pd.Timestamp(Date_output),
                                                    offer = hitpath_offer_ID,
                                                    P_drop_dict = Account_P_Max,
                                                    eligibility_df = CT_eligibility_df)
                cobra_output = get_cobra_slot(cobra = cobra,
                                        account = DPPub,
                                        date = pd.Timestamp(Date_output),
                                        offer = hitpath_offer_ID,
                                        Drop_Number = Drop_Number_output,
                                        CCIDs = CCIDs_to_Test)  

                #Drop_Number_output = 1
                Time_output = cobra_output[0]
                Segment = cobra_output[1]
                Send_Strategy_output = 'CT'
                Hitpath_OfferID_output = hitpath_offer_ID
                #Scheduling_Name_output = offer_wk[offer_wk['Hitpath Offer ID'] == hitpath_offer_ID]['Scheduling Name'].iloc[0] # NEED TO CHANGE TO SMS 
                Offer_output = cobra_output[2]
                Limit_output = cobra_output[3]
                Offset_output = cobra_output[4]
                Creative_output = cobra_output[5]
                Job_Name_output = cobra_output[6] 
                if cobra_output[7] == 0:
                    drop_date_info[1] = 0 
                Eligibility_output = + drop_date_info[1]
                
                # Update Cobra 
                ## so it does not repeat the same accounts if it's scheduling multiple batches of the same offer
                cobra_row = pd.DataFrame({'Date':[Date_output],
                                        'shortcode_DP.SV': [DPPub_output],
                                        'Hitpath Offer ID': [hitpath_offer_ID],
                                        'Campaign ID':[Offer_output],
                                        'Send Strategy': [Send_Strategy_output],
                                        'Drop': [Drop_Number_output]
                                        })
                
                cobra.loc[((cobra['shortcode_DP.SV']==DPPub_output) & (cobra['Date']==Date_output) & (cobra['Drop'] ==Drop_Number_output )), 'Hitpath Offer ID'] = hitpath_offer_ID

                drop_row = [[Date_output, DPPub_output, Drop_Number_output, Time_output, Segment, Send_Strategy_output,
                            Offer_output, Limit_output, Offset_output, Creative_output, Job_Name_output, Eligibility_output]]
                
                upcoming_CT_schedule = upcoming_CT_schedule._append(drop_row, ignore_index=True)
                
                success_counter = success_counter + drop_date_info[1]
                if success_counter == 2:
                    print("** We have found", success_counter, "CTs", "ending search for", hitpath_offer_ID, "**")
                    print(" ")
                    print(" ")
                    break
                    
            except:
                print("* FAILED ON", DPPub, "*")
            
 
    # Rename the columns
    #upcoming_CT_schedule.columns = columns 
upcoming_CT_schedule = upcoming_CT_schedule.rename(columns=dict(zip(upcoming_CT_schedule.columns, columns)))
upcoming_CT_schedule = upcoming_CT_schedule[upcoming_CT_schedule['Can it Test?'] == 1]      
upcoming_CT_schedule = upcoming_CT_schedule.drop('Can it Test?', axis=1)


    #CT_pipeline_output = prep_for_CT_pipeline_smartsheet(df = upcoming_CT_schedule, offer_wk = offer_wk)
upcoming_CT_schedule.to_csv('Content Testing Schedule.csv', index=False)  
    #CT_pipeline_output.to_csv('Content Testing Pipeline Output.csv', index=False)

    # upload content testing schedule to Google Sheet


print(bold + "CCID SCHEDULE GENERATION IS DONE, PLEASE CHECK OUTPUT BEFORE RUNNING SCHEDULE CODE.")



Mamba are loaded.
Starting process to create CCID testing schedule.
------------------------------------------------------------------------
Generating Schedule for CCIDs using Hitpath Offer ID: 12089
------------------------------------------------------------------------
12089 None
------------------------------------------------------------------------
Generating Schedule for CCIDs using Hitpath Offer ID: 12089
------------------------------------------------------------------------
12089 None
------------------------------------------------------------------------
Generating Schedule for CCIDs using Hitpath Offer ID: 12113
------------------------------------------------------------------------
12113 Loan
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
Searching for available date for FLC_EDM.247L
     No existing P drop, we will add in a drop.
     The earliest available week to test with proper gapping is 2024-03-29 00:00:00
     Earliest available drop is 

* FAILED ON HZB_AMD.PL *
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
Searching for available date for HZB_FSM.YS
     No existing P drop, we will add in a drop.
     The earliest available week to test with proper gapping is 2024-03-21 00:00:00
* FAILED ON HZB_FSM.YS *
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
Searching for available date for HZB_B2.F
     No existing P drop, we will add in a drop.
     The earliest available week to test with proper gapping is 2024-03-20 00:00:00
* FAILED ON HZB_B2.F *
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
Searching for available date for HZB_PA.PS
     No existing P drop, we will add in a drop.
     The earliest available week to test with proper gapping is 2024-03-28 00:00:00
* FAILED ON HZB_PA.PS *
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
Searching for available date for HZB_AL.PL.3
     No existing P drop, we will add in a drop.
 

In [31]:
upcoming_CT_schedule

,Date,Affiliate ID_DP.DS,Drop Number,Time,Segment,Send Strategy,Offer,Limit,Offset,Creative,Job Name
0,2024-03-29,FLC_EDM.247L,2,10:00 AM PST,FLC_EDM.247L_30DC_NEXL,CT,12113 - AVL SL FLC,3500.0,0,12113.SC.FLC.453521,SS_FLC_EDM-247L-30DC-NEXL_12113_CT_29Mar24
1,2024-03-21,FLC_PN.SWP,2,10:00 AM PST,FLC_PN.SWP_37DC,CT,12113 - AVL SL FLC,6000.0,0,12113.SC.FLC.453521,SS_FLC_PN-SWP-37DC_12113_CT_21Mar24


In [32]:
""" 
for index, row in content_feedback_report.iterrows():

    # variables
    hitpath_offer_ID  = content_feedback_report['HitPath Offer ID'].iloc[index]
    CCIDs  = content_feedback_report["Content ID's"].iloc[index]
    OfferType = content_feedback_report["Offer Type"].iloc[index]
    Days_to_test = content_feedback_report["Days to Test"].iloc[index]

    print("------------------------------------------------------------------------")
    print("Generating Schedule for CCIDs using Hitpath Offer ID:", hitpath_offer_ID)
    print("------------------------------------------------------------------------")
    
    offer_gaps_df = get_offer_vertical_gaps(df = lexi, EIMT = EIMT)
    account_rankings_df = get_account_rankings(df = lexi, offer = hitpath_offer_ID)
    DPPub_test = list(account_rankings_df['DP&Pub'])
    
    success_counter = 0
    for DPPub in DPPub_test:

        try:
            CT_eligibility_df = check_CT_eligiblity(CT_settings_url = 'https://docs.google.com/spreadsheets/d/1TmWNe9MYmAB9s2MyH0UOOS1ZKdJKYl0AY6jgL6grHcc/edit#gid=1087982961',
                                                 default_max_drops = 3,
                                                 lexi = lexi,
                                                 cobra = cobra,
                                                 dataset_convertor = Dataset_to_DPPub,
                                                 offer = hitpath_offer_ID)
            
        except:
            print("* FAILED ON", DPPub, "*")
            pass
"""

' \nfor index, row in content_feedback_report.iterrows():\n\n    # variables\n    hitpath_offer_ID  = content_feedback_report[\'HitPath Offer ID\'].iloc[index]\n    CCIDs  = content_feedback_report["Content ID\'s"].iloc[index]\n    OfferType = content_feedback_report["Offer Type"].iloc[index]\n    Days_to_test = content_feedback_report["Days to Test"].iloc[index]\n\n    print("------------------------------------------------------------------------")\n    print("Generating Schedule for CCIDs using Hitpath Offer ID:", hitpath_offer_ID)\n    print("------------------------------------------------------------------------")\n    \n    offer_gaps_df = get_offer_vertical_gaps(df = lexi, EIMT = EIMT)\n    account_rankings_df = get_account_rankings(df = lexi, offer = hitpath_offer_ID)\n    DPPub_test = list(account_rankings_df[\'DP&Pub\'])\n    \n    success_counter = 0\n    for DPPub in DPPub_test:\n\n        try:\n            CT_eligibility_df = check_CT_eligiblity(CT_settings_url = \'https: